In [15]:
import json

from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset

## Demo RoSummary

### Load AlephNews dataset

In [5]:
dataset = load_dataset("readerbench/AlephNews")

Using custom data configuration readerbench--AlephNews-df8c20c4b522d82e


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/mihai/.cache/huggingface/datasets/readerbench___json/readerbench--AlephNews-df8c20c4b522d82e/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['url', 'title', 'summary', 'paragraphs'],
        num_rows: 38683
    })
    test: Dataset({
        features: ['url', 'title', 'summary', 'paragraphs'],
        num_rows: 2143
    })
    validation: Dataset({
        features: ['url', 'title', 'summary', 'paragraphs'],
        num_rows: 2036
    })
})


In [16]:
print(json.dumps(dataset['test'][0], indent=4, ensure_ascii=False))

{
    "url": "https://alephnews.ro/entertainment/boema-lui-puccini-pe-timp-de-pandemie-primul-spectacol-de-opera-live-drive-in-din-europa/",
    "title": "\"Boema\" lui Puccini, pe timp de pandemie. Primul spectacol de operă live drive-in din Europa.",
    "summary": [
        "Pandemia a dus anul ăsta la o premieră în lumea spectacolelor: opera drive-in.",
        "\"Boema\" lui Puccini s-a jucat pentru prima dată într-o parcare şi a fost urmărită din maşini.",
        "Opera de la Bucureşti şi-a redeschis porţile şi te aşteaptă la spectacole."
    ],
    "paragraphs": [
        "Englezilor le place să facă lucrurile altfel. Și încearcă să nu te dezamăgească niciodată. Primul spectacol de operă live drive-in din Europa a avut loc în weekend la Londra.S-a jucat \"Boema\" lui Puccini, în parcarea de la Alexandra Palace.Sala de evenimente, faimoasă în oraş, e închisă încă de la începutul pandemiei.Așa că spectacolul a fost adaptat vremurilor. Spectatorii au stat în mașini și au fost serv

### Load model

In [9]:
model = TFGPT2LMHeadModel.from_pretrained("readerbench/RoSummary-medium")
tokenizer = GPT2Tokenizer.from_pretrained("readerbench/RoSummary-medium")

2022-10-31 14:06:19.712821: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 14:06:19.736844: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 14:06:19.737579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 14:06:19.739137: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

### Generate summary for news

In [10]:
index = 0
config_gen = 'beam-search-4' # greedy beam-search-4 sample-top-p
item = dataset['test'][index]
configs_gen = {
        'greedy': {},
        'beam-search-4': {'num_beams': 4, 'early_stopping': True},
        'sample-top-p': {'top_k': 25, 'top_p': 0.94, 'do_sample': True},
}

In [11]:
sentence_input = f'Text: {item["title"] + " ".join(item["paragraphs"])} Summary:'
sentence_token = tokenizer.encode(sentence_input, return_tensors='tf')
len_input_tokens = len(sentence_token[0])

predict_tokens = model.generate(sentence_token, max_length=1024, pad_token_id=tokenizer.eos_token_id,
                                **configs_gen["beam-search-4"])[0][len_input_tokens:]
predict_text = tokenizer.decode(predict_tokens).replace('<|endoftext|>', '')

#### Original summary

In [12]:
" ".join(item["summary"])

'Pandemia a dus anul ăsta la o premieră în lumea spectacolelor: opera drive-in. "Boema" lui Puccini s-a jucat pentru prima dată într-o parcare şi a fost urmărită din maşini. Opera de la Bucureşti şi-a redeschis porţile şi te aşteaptă la spectacole.'

#### Generated summary

In [13]:
predict_text

' Primul spectacol de operă live drive-in din Europa a avut loc în weekend la Londra. Spectatorii au stat în mașini și au fost serviţi cu îngheţată, ciocolată şi băuturi. Opera de la Bucureşti te aşteaptă şi ea din nou la spectacole începând din weekendul ăsta, după mai bine de şase luni de pauză.'